<a href="https://colab.research.google.com/github/cmunozperez/NLP-Python-2025/blob/main/Notebook_3_Naive_Bayes_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicciones con Naive Bayes

Empezamos realizando algunas importaciones.

- Regex (o expresiones regulares) es una herramienta para buscar, extraer o reemplazar texto usando patrones. Se usa con el módulo re en Python.
- spaCy es una biblioteca de procesamiento de lenguaje natural (NLP) en Python. Permite analizar textos: separar palabras, identificar entidades (como nombres de personas, lugares), etc.

In [ ]:
import re # regex
import spacy # Para preprocesar los datos

!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Vamos a utilizar el modelo "grande" de spaCy para el inglés. Es bastante más lento que los modelos pequeños, pero no es algo que vaya a afectarnos demasiado para el preprocesamiento.

In [ ]:
nlp = spacy.load("en_core_web_lg")

ej1 = nlp("This study deals with syntactic structure both in the broad sense (as opposed to semantics) and the narrow sense (as opposed to phonemics and morphology).")

for token in ej1:
    print(token.text)

This
study
deals
with
syntactic
structure
both
in
the
broad
sense
(
as
opposed
to
semantics
)
and
the
narrow
sense
(
as
opposed
to
phonemics
and
morphology
)
.


Como vimos ayer, podemos obtener automáticamente la categoría de las palabras.

In [ ]:
for token in ej1:
    print(f"{token.text} -> {token.pos_}")

This -> DET
study -> NOUN
deals -> VERB
with -> ADP
syntactic -> ADJ
structure -> NOUN
both -> PRON
in -> ADP
the -> DET
broad -> ADJ
sense -> NOUN
( -> PUNCT
as -> SCONJ
opposed -> VERB
to -> ADP
semantics -> NOUN
) -> PUNCT
and -> CCONJ
the -> DET
narrow -> ADJ
sense -> NOUN
( -> PUNCT
as -> SCONJ
opposed -> VERB
to -> ADP
phonemics -> NOUN
and -> CCONJ
morphology -> NOUN
) -> PUNCT
. -> PUNCT


También podemos hacer *Named Entity Recognition* (NER), i.e., una técnica de procesamiento de lenguaje natural que identifica y clasifica entidades en un texto de acuerdo a una tipología.

- PERSON: gente real o no
- NORP: nacionalidades o grupos políticos, religiosos, etc.
- FAC: edificioes, aeropuertos, calles, etc.
- ORG: Empresas, instituciones
- GPE: ciudades, provincias, países
- Otras...

In [ ]:
ej2 = nlp("Lucio Fulci, an Italian film director known for his work in horror cinema, was born in Rome in 1927 and directed cult classics like The Beyond and Zombie Flesh Eaters.")

for ent in ej2.ents:
    print(f"{ent.text} -> {ent.label_}")

Lucio Fulci -> PERSON
Italian -> NORP
Rome -> GPE
1927 -> DATE
The Beyond and Zombie Flesh Eaters -> WORK_OF_ART


Esta celda contiene la funcionalidad que vamos a utilizar: la lematización. Sirve para obtener ``la forma de diccionario'' de una palabra, para facilitar establecer similitudes entre los textos.

In [ ]:
ej3 = nlp("The children ran to the geese, who had eaten the thieves' loaves, while the mice hid from the teeth of the wolves.")

for token in ej3:
    print(f"{token.text} -> {token.lemma_}")

The -> the
children -> child
ran -> run
to -> to
the -> the
geese -> goose
, -> ,
who -> who
had -> have
eaten -> eat
the -> the
thieves -> thief
' -> '
loaves -> loaf
, -> ,
while -> while
the -> the
mice -> mouse
hid -> hide
from -> from
the -> the
teeth -> tooth
of -> of
the -> the
wolves -> wolf
. -> .


Para lematizar, vamos a usar la siguiente función.

In [ ]:
def lemmatize(text):
    doc = nlp(text.lower())
    return ' '.join([token.lemma_ for token in doc if not token.is_punct and not token.is_space])

In [ ]:
ejemplo = lemmatize('This is an English sentence that needs to be analyzed.')
ejemplo

'this be an english sentence that need to be analyze'

## Generando el DataFrame para training

Antes de entrenar un modelo de Naive Bayes, es necesario transformar el DataFrame porque el modelo no puede trabajar directamente con texto crudo o datos categóricos. Además de lematizar, los textos deben convertirse en vectores numéricos (por ejemplo, usando CountVectorizer) que representen la frecuencia o importancia de las palabras.

Esto abre el archivo CSV.



In [ ]:
csv = "https://raw.githubusercontent.com/cmunozperez/lingbuzz_data_analysis/refs/heads/main/lingbuzz_002_007537.csv"


Lo llevamos a un DataFrame

In [ ]:
import pandas as pd

df = pd.read_csv(csv)

Limpiamos NaNs.

In [ ]:
df.dropna(subset=['Title'], inplace=True)

df['Abstract'] = df['Abstract'].fillna('')
df['Keywords'] = df['Keywords'].fillna('')

Cambiamos el formato de las fechas.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%B %Y')
df['Date'] = df['Date'].dt.to_period('M')

*One-hot encoding* es una técnica para convertir variables categóricas (e.g., la disciplina a la que pertenece un manuscrito) en una representación numérica binaria. Cada categoría se transforma en una nueva columna que toma el valor 1 si el registro pertenece a esa categoría, o 0 si no.

In [ ]:
disciplines = ['phonology', 'morphology', 'syntax', 'semantics']

for word in disciplines:
  df[word] = df['Keywords'].apply(lambda x: 1 if word in x else 0)

Combinamos el abstract con su título en una única celda a la que llamaremos "Text". Esta es la variable *x* que utilizaremos para predecir la etiqueta *y* (la disciplina).

In [ ]:
df['Text'] = df['Title'] + '. ' + df['Abstract']

Pregunta: ¿qué pasa si nuestra muestra tiene muchas más muestras de una clase que de otras? Por ejemplo, si entrenamos un clasificador que reconozca spam, y lo entrenamos con 100.000 mensajes normales y solo 1000 mensajes de spam, ¿qué pasaría?

In [ ]:
label_count = {dis: df[dis].sum() for dis in disciplines}
label_count

{'phonology': np.int64(1359),
 'morphology': np.int64(1995),
 'syntax': np.int64(5215),
 'semantics': np.int64(2769)}

Establecemos una variable con el número *n* de entradas para la disciplina menos representada en el repositorio.

In [ ]:
lowest_label_count = min(label_count.values())

Y tomamos las *n* entradas más recientes de cada disciplina.

In [ ]:
selected_abstracts = []

for dis in disciplines:
    small_df = df[df[dis] == 1]
    small_df = small_df.sort_values(by='Date', ascending=False)
    small_df = small_df.head(lowest_label_count)
    selected_abstracts.append(small_df)

Combinamos nuevamente en un DataFrame df. Aprovechemos también para deshacernos de la información que no vamos a utilizar en el entrenamiento (lugar de publicación, id, cantidad de descargas, etc.).

In [ ]:
df = pd.concat(selected_abstracts, ignore_index=False, join='inner')

df = df[['Text', 'phonology', 'morphology', 'syntax', 'semantics']]
df

,Text,phonology,morphology,syntax,semantics
7295,Les langues des signes : en France et à trave...,1,0,1,1
6941,Review of: [Polinsky et al] Oxford Handbook of...,1,1,1,1
7138,Prosodic strength in Campidanese Sardinian as ...,1,0,0,0
6991,A Neo-Trubetzkoyan approach to phonotactic lea...,1,0,0,0
7346,What phonology is and why it should be. This c...,1,0,0,0
...,...,...,...,...,...
3512,Genericity in event semantics: A look at Yorub...,0,0,0,1
4082,Modes of presentation in attitude reports. In ...,0,0,0,1
3686,Justifications for a discontinuity theory of l...,0,0,1,1
4162,Reducing coreference to co-binding. The paper ...,0,0,1,1


Y finalmente aplicamos la lematización sobre la columna "Text". Esto tarda cuatro minutos veintisiete segundos.

In [ ]:
df['Text'] = df['Text'].apply(lemmatize)

##

## Entrenando el modelo

Empecemos con las importaciones necesarias para el entrenamiento.

- train_test_split: divide los datos en conjuntos de entrenamiento y prueba.

- CountVectorizer: convierte texto en una matriz de conteo de palabras (bolsa de palabras).

- OneVsRestClassifier: es un wrap que permite entrenar un clasificador con etiquetas múltiples.

- MultinomialNB: clasificador Naive Bayes.

- classification_report: muestra métricas como precision, recall y F1 para evaluar el modelo.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

El texto de cada artículo constituye los "datos de entrada". Lo que queremos es que nos prediga la disciplina.

In [ ]:
X = df['Text']
y = df[['phonology', 'morphology', 'syntax', 'semantics']]

Aplicamos el vectorizador. Dejamos de lado las *stopwords* típicas del inglés. Además, ignoramos palabras que aparezcan en el 75% de los textos (y que, por tanto, no permitan distinguirlos).

In [ ]:
vectorizer = CountVectorizer(stop_words='english', max_df=0.75)
X_vec = vectorizer.fit_transform(X)

Dividimos los datos de entrenamiento y prueba.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

Y entrenamos nuestro clasificador.

In [ ]:
clf = OneVsRestClassifier(MultinomialNB())
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=MultinomialNB())

## Probando el modelo

In [ ]:
y_pred = clf.predict(X_test)

print("Classification Report:\n", classification_report(y_test, y_pred, target_names=y.columns))

Classification Report:
               precision    recall  f1-score   support

   phonology       0.88      0.91      0.90       419
  morphology       0.82      0.90      0.86       515
      syntax       0.91      0.93      0.92       721
   semantics       0.85      0.88      0.87       498

   micro avg       0.87      0.91      0.89      2153
   macro avg       0.87      0.90      0.89      2153
weighted avg       0.87      0.91      0.89      2153
 samples avg       0.89      0.92      0.88      2153



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Bueno, eso es todo: nuestro modelo está terminado. Lo único que necesitamos es una manera de usarlo con abstracts nuevos que queramos categorizar. Podemos escribir una función para esto.

In [ ]:
def classify_text_input(clf, vectorizer):

    text = input("Dame un abstract de lingüística en inglés:\n").strip()
    if not text:
        print("No me diste un abstract")


    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc if not token.is_punct and not token.is_space])

    X_vec = vectorizer.transform([lemmatized_text])

    prediction = clf.predict(X_vec)[0]

    labels = ['phonology', 'morphology', 'syntax', 'semantics']
    #print(zip(labels,prediction))

    predicted_labels = [label for label, val in zip(labels, prediction) if val == 1]

    if predicted_labels:
        print("Disciplina:", ", ".join(predicted_labels))
    else:
        print("Ninguna.")

    return predicted_labels

Acá probamos la función con el clasificador y el vectorizador que ya instanciamos.

In [ ]:
classify_text_input(clf, vectorizer)

Dame un abstract de lingüística en inglés:
pedro fue al teatro ayer
Disciplina: morphology, syntax, semantics


['morphology', 'syntax', 'semantics']

# LingPeer

En su base, LingPeer no difiere demasiado de lo que acabamos de hacer. La distinción relevante a nuestros fines es que

- en vez de utilizar el abstract para predecir a qué disciplina pertenece,
- lo usa para intentar predecir quién es su autor.

No vamos a hacer todos los pasos de preprocesamiento nuevamente. Solo hay uno que creo es importante destacar: el reemplazo de keywords de más de una palabra por un placeholder (ver slide correspondiente) .

In [ ]:
import joblib

# Download the file using wget
!wget -O n_grams.pkl https://github.com/cmunozperez/LingPeer/raw/refs/heads/master/LingPeer/n_grams.pkl

# Load the file
n_grams = joblib.load("n_grams.pkl")

--2025-07-30 22:49:04--  https://github.com/cmunozperez/LingPeer/raw/refs/heads/master/LingPeer/n_grams.pkl
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cmunozperez/LingPeer/refs/heads/master/LingPeer/n_grams.pkl [following]
--2025-07-30 22:49:04--  https://raw.githubusercontent.com/cmunozperez/LingPeer/refs/heads/master/LingPeer/n_grams.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 189231 (185K) [application/octet-stream]
Saving to: ‘n_grams.pkl’

n_grams.pkl         100%[===================>] 184.80K  --.-KB/s    in 0.03s   

2025-07-30 22:49:04 (5.46 MB/s) - ‘n_grams.pkl’ saved [1892

In [ ]:
n_grams

{'head marking': 'ngram0',
 'dependent mark': 'ngram1',
 'extended projection': 'ngram2',
 'double access reading': 'ngram3',
 'sequence of tense': 'ngram4',
 'tense agreement': 'ngram5',
 'tense computation': 'ngram6',
 'the syntax pf correlation': 'ngram7',
 'mandarin chinese': 'ngram8',
 'gradable predicate': 'ngram9',
 'degree abstraction': 'ngram10',
 'degree comparison': 'ngram11',
 'that less clause': 'ngram12',
 'dynamic phase': 'ngram13',
 'prosodic morphology': 'ngram14',
 'optimality theory': 'ngram15',
 'wh movement': 'ngram16',
 'wh morphology': 'ngram17',
 'complementizer agreement': 'ngram18',
 'subject non subject asymmetry': 'ngram19',
 'feature co occurrence': 'ngram20',
 'complementizer allomorphy': 'ngram21',
 'nominal parameter': 'ngram22',
 'pro drop': 'ngram23',
 'partial null subject language': 'ngram24',
 'rich agreement null subject language': 'ngram25',
 'discourse pro drop language': 'ngram26',
 'semi pro drop language': 'ngram27',
 'null np anaphora': 'ngra

La siguiente es una función que reemplaza los n-grams en un texto por su correspondiente placeholder. No la vamos a correr. La pongo para que vean cómo funciona.

In [ ]:
def replace_ngrams(cell):

    '''
    This replaces the n-gram keywords in the abstract.
    '''

    for ngram, dummy_token in n_gram_dict.items():

        # This finds the ngrams in the text
        pattern = r'\b' + re.escape(ngram) + r'\b'

        # And this replaces the ngrams with a token
        cell = re.sub(pattern, dummy_token, cell, flags=re.IGNORECASE)

    return cell

Vamos a importar uno de los modelos con los que funciona *LingPeer* para probarlo, junto con su correspondiente vectorización.

In [ ]:
!wget -O classifier1.pkl https://github.com/cmunozperez/LingPeer/raw/refs/heads/master/LingPeer/classifier1.pkl
classifier1 = joblib.load("classifier1.pkl")

!wget -O c_vect1.pkl https://github.com/cmunozperez/LingPeer/raw/refs/heads/master/LingPeer/c_vect1.pkl
c_vect1 = joblib.load("c_vect1.pkl")

--2025-07-30 22:49:51--  https://github.com/cmunozperez/LingPeer/raw/refs/heads/master/LingPeer/classifier1.pkl
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cmunozperez/LingPeer/refs/heads/master/LingPeer/classifier1.pkl [following]
--2025-07-30 22:49:51--  https://raw.githubusercontent.com/cmunozperez/LingPeer/refs/heads/master/LingPeer/classifier1.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43164143 (41M) [application/octet-stream]
Saving to: ‘classifier1.pkl’

classifier1.pkl     100%[===================>]  41.16M   159MB/s    in 0.3s    

2025-07-30 22:49:51 (159 MB/s) - ‘classifi

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MultinomialNB from version 1.2.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


302 Found
Location: https://raw.githubusercontent.com/cmunozperez/LingPeer/refs/heads/master/LingPeer/c_vect1.pkl [following]
--2025-07-30 22:49:52--  https://raw.githubusercontent.com/cmunozperez/LingPeer/refs/heads/master/LingPeer/c_vect1.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 334807 (327K) [application/octet-stream]
Saving to: ‘c_vect1.pkl’

c_vect1.pkl         100%[===================>] 326.96K  --.-KB/s    in 0.04s   

2025-07-30 22:49:52 (7.15 MB/s) - ‘c_vect1.pkl’ saved [334807/334807]



/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.2.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


La siguiente es la función que pone en funcionamiento el modelo. Básicamente, toma un abstract y devuelve una lista ordenada de autores de acuerdo al "puntaje" que les asigna el clasificador naive Bayes.

In [ ]:
def model1(abstract):
    # This vectorizes the provided abstract
    abstract_vect = c_vect1.transform([abstract])

    # This gets the probabilities for each of the authors in the database
    pred_proba = classifier1.predict_proba(abstract_vect.reshape(1, -1))

    class_labels = classifier1.classes_

    class_labels = [str(i) for i in class_labels]

    # This combines the probabilities with each of the authors
    class_probabilities = [(label, prob) for label, prob in zip(class_labels, pred_proba[0])]

    # Sort class probabilities in descending order
    class_probabilities.sort(key=lambda x: x[1], reverse=True)

    return class_probabilities

Hagamos una función para probar este modelo de *LingPeer*. La idea es obtener los diez mejores revisores en el repositorio para un cierto abstract.

In [ ]:
def probar_lingpeer():
    abstract = input("Dame un abstract y te tiro diez nombres de revisores: ")
    predict = model1(abstract)
    return predict[:10]

In [ ]:
probar_lingpeer()

Dame un abstract y te tiro diez nombres de revisores: This paper examines metaphor through the lens of Conceptual Blending Theory, as developed by Fauconnier and Turner. Unlike more traditional approaches such as Conceptual Metaphor Theory (Lakoff & Johnson), which map elements between source and target domains, Conceptual Blending emphasizes the dynamic integration of multiple mental spaces to create novel meaning. In the case of metaphor, this theory explains how elements from different domains combine within an emergent blended space, producing interpretations that are creative and context-sensitive. Drawing on examples from contemporary Spanish, the study demonstrates how conceptual blending operates not only in conventional metaphors (time is money), but also in novel and humorous expressions, revealing the cognitive flexibility underlying metaphorical language. This approach offers a more nuanced account of the mental processes involved in metaphor construction and interpretation

[('Prakash Mondal', np.float64(0.5314076273744693)),
 ('Ailís Cournane', np.float64(0.08241300247311187)),
 ('Canaan Breiss', np.float64(0.07574595057197107)),
 ('Roni Katzir', np.float64(0.06720091476867822)),
 ('Itamar Kastner', np.float64(0.04954090363291377)),
 ('Deniz Satik', np.float64(0.044046661283897615)),
 ('Friederike Moltmann', np.float64(0.04179008641720001)),
 ('Jianrong Yu', np.float64(0.01246300240903974)),
 ('Linmin Zhang', np.float64(0.01051706879394939)),
 ('Martin Haspelmath', np.float64(0.010007492090832102))]